# Computer Simulation Project
Simulating a discrete-event CPU scheduler.

**Authors**: <br>
Mohammadreza Mofayezi 98106059 <br>
Amirhossein Bagheri 98105621

## Import Libraries

In [81]:
import simpy
from typing import List

import pandas as pd
import numpy  as np
from queue import PriorityQueue

from scipy.stats import norm, expon, poisson

## Inputs
The parameters, number of processes and duration of simulation.

In [79]:
# parameters
X = 0.2
Y = 3
Z = 10
# number of processes
N = 50
# length of simulation
L = 600
# task threshold
k = 5
seed = 1234
T1 = 2
T2 = 2

In [4]:
np.random.seed(seed=seed)

## Job Creator

In [84]:
class Task:
    def __init__(self, id: int):
        self.id = id
        self.inter_arrival = int(poisson.rvs(X))
        self.service_time = int(expon.rvs(Y))
        self.remaining_service = self.service_time
        self.priority = self.get_priority()
        self.arrival_time = None
        self.finish_time = None

    def get_priority(self):
        return np.random.choice([1, 2, 3], 1, p=[0.1, 0.2, 0.7])[0]

    def __eq__(self, __o: object) -> bool:
        return self.priority == __o.priority

    def __repr__(self) -> str:
        return f'Task {self.id} - InterArrival: {self.inter_arrival} - ServiceTime: {self.service_time}'

In [85]:
class JobCreator:
    def __init__(self, num_tasks: int):
        self.num_tasks = num_tasks
        self.tasks = PriorityQueue()
        self.all_tasks = []

    def run(self):
        start = 0
        task = Task(0)
        task.arrival_time = 0
        self.tasks.put((task.arrival_time,task.priority, -task.remaining_service, task))
        self.all_tasks.append(task)
        
        for i in range(1,self.num_tasks):
            task = Task(i)
            start += task.inter_arrival
            task.arrival_time = start
            self.tasks.put((task.arrival_time,task.priority, -task.remaining_service, task))
            self.all_tasks.append(task)

    def get_task(self,time: int):
        if self.tasks.queue[0][0] <= time:
            return self.tasks.get()[-1]
        return None
    
    def empty_job(self):
        return self.tasks.empty()


In [89]:
class Que:
    def __init__(self):
        self.q = []
    
    def insert(self,task: Task):
        self.q.insert(0,task)
    
    def __len__(self):
        return len(self.q)
    
    def run(self,time: int):
        pass

class RR(Que):
    def __init__(self, T: int):
        super().__init__()
        self.T = T

class Processor:
    def __init__(self, num_tasks: int, maxsimul: int, queues : List):
        self.Job_creator = JobCreator(num_tasks)
        self.Job_creator.run()
        self.queues = queues
        self.simultime = 0
        self.max_simultime = maxsimul
    
    def job_loader(self, k: int):
        queue = PriorityQueue()
        for _ in range(k):
            if self.Job_creator.empty_job():
                break
            task = self.Job_creator.get_task(self.simultime)
            if task is None:
                break

            queue.put((task.priority,-task.remaining_service,task))
        
        for q in queue.queue:
            self.queues[0].insert(q[-1])
    
    def run(self):
        pass


    def dispatcher(self):
        pass

processor = Processor(N, L, [RR(T1),RR(T2),Que()])


        

In [67]:
creator = JobCreator(N)
creator.run()
creator.get_task()

Task 40 - InterArrival: 0 - ServiceTime: 7

## Scheduler

In [25]:
class Scheduler:
    def __init__(self, env):
        self.env = env
        simpy.Resource(env, )

    def job_loader(self):
        pass

    def dispatcher(self):
        pass

1